# Text Classification
 - [Introductory tutorial](https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a)

### Multi-Label Classification
 - [Sklearn package](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html)
 - [Kaggle examples of common methods](https://www.kaggle.com/reiinakano/basic-nlp-bag-of-words-tf-idf-word2vec-lstm)

### Text-Classification using word2vec
 - [word2vec tutorial](http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/)<br>
 - [deep learning turoial](https://datawarrior.wordpress.com/2016/10/12/short-text-categorization-using-deep-neural-networks-and-word-embedding-models/)

In [1]:
import os
import sys
import re
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
path = os.getcwd()

# Data Sources
 - [NLTK Reuters data](https://miguelmalvarez.com/2015/03/20/classifying-reuters-21578-collection-with-python-representing-the-data/)


In [3]:
## Download Reuters Data
## https://archive.ics.uci.edu/ml/datasets/Reuters-21578+Text+Categorization+Collection
# !curl -O https://archive.ics.uci.edu/ml/machine-learning-databases/reuters21578-mld/reuters21578.tar.gz
# !tar xzvf reuters21578.tar.gz

 - [20 News Group data](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html)

In [4]:
# Train & Test data
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=False)
twenty_test = fetch_20newsgroups(subset='test', shuffle=False)

twenty_train.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [5]:
names = {e:v for e,v in enumerate(twenty_train['target_names'])}

In [6]:
len(twenty_train['data']), len(twenty_train['target']), 

(11314, 11314)

In [7]:
n = 0
def printArticle(n):
    global names, twenty_train
    target = twenty_train['target'][n]
    name = names[target]
    doc =  twenty_train['data'][n]
    print('{}\n{}'.format(name,doc))
    
printArticle(n)

rec.sport.baseball
From: cubbie@garnet.berkeley.edu (                               )
Subject: Re: Cubs behind Marlins? How?
Article-I.D.: agate.1pt592$f9a
Organization: University of California, Berkeley
Lines: 12
NNTP-Posting-Host: garnet.berkeley.edu


gajarsky@pilot.njin.net writes:

morgan and guzman will have era's 1 run higher than last year, and
 the cubs will be idiots and not pitch harkey as much as hibbard.
 castillo won't be good (i think he's a stud pitcher)

       This season so far, Morgan and Guzman helped to lead the Cubs
       at top in ERA, even better than THE rotation at Atlanta.
       Cubs ERA at 0.056 while Braves at 0.059. We know it is early
       in the season, we Cubs fans have learned how to enjoy the
       short triumph while it is still there.



In [8]:
# Remove non alpha-numerical chars, lowercase, strip whitespace
.replace('[^a-zA-Z0-9 ]+', ' ', regex=True).str.lower().str.strip()

SyntaxError: invalid syntax (<ipython-input-8-c8756b08e8d0>, line 2)

# Classification
[Classification reports](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report)

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [10]:
# def sample(data):
#     X_train, X_test,\
#     y_train, y_test = train_test_split(data['data'],
#                                        data['target'],
#                                        test_size=0.5,
#                                        stratify=data['target'])
#     return y_train

In [11]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
CV = CountVectorizer(strip_accents='unicode',
                     lowercase=True,
                     stop_words='english',)

# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
TF = TfidfTransformer()

# Naive Bayes

In [18]:
# model param
NB_params = {'vect__ngram_range':[(1,1),(1,2)],
             'vect__analyzer':['word','char'],
             'tfidf__use_idf':('True'),
             'model__alpha':(1e-1, 1e-3)}

NB_pipe = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('model', MultinomialNB())])

gs_NB = GridSearchCV(estimator=NB_pipe,
                          param_grid=NB_params,
                          n_jobs=-1,
                          cv=4)

%time gs_NB = gs_NB.fit(twenty_train['data'][:5000], twenty_train['target'][:5000])

Wall time: 4min 14s


In [20]:
print(gs_NB.best_score_, gs_NB.best_params_,'\n')
NBcv_predict = gs_NB.predict(twenty_test['data'])
print('NB-CV accuracy: %.3f\n' %np.mean(NBcv_predict == twenty_test['target']))
print(metrics.classification_report(twenty_test['target'],NBcv_predict))

NB-CV accuracy: 0.792


# SVM & Logit

In [ ]:
# model param
SVM_params = {'vect__ngram_range':[(1,1),(1,2)],
              'vect__analyzer':['word','char'],
              'tfidf__use_idf':('True'),
              'model__loss':('hinge','log'),
              'model__alpha':(1e-2, 1e-3),
              'model__penalty': ('l2', 'elasticnet'),
              'model__max_iter': (50, 100)}

SVM_pipe = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('model',SGDClassifier()),])

gs_SVM = GridSearchCV(SVM_pipe,
                      param_grid=SVM_params,
                      n_jobs=-1,
                      cv=4)

%time gs_SVM = gs_SVM.fit(twenty_train['data'][:5000], twenty_train['target'][:5000])

C:\ProgramData\Anaconda3\envs\nltk\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
print(gs_SVM.best_score_, gs_SVM.best_params_)
SVMcv_predict = gs_SVM.predict(twenty_test['data'])
print('SVM-CV accuracy: %.3f' %np.mean(SVMcv_predict, twenty_test['target']))
print(metrics.classification_report(twenty_test['target'], SVMcv_predict)

# Logit

In [ ]:
# LOG_params = {'vect__ngram_range':[(1,1),(1,2)],
#               'tfidf__use_idf':('True','False'),
#               'model__alpha':(1e-1, 1e-3)}

# LOG_pipe = Pipeline([('vect', CountVectorizer()),
#                      ('tfidf', TfidfTransformer()),
#                      ('model',SGDClassifier(loss='log', penalty='l2',alpha=1e-3, max_iter=5, random_state=42)),])

# gs_LOG = GridSearchCV(LOG_pipe, param_grid=LOG_params, n_jobs=-1)
# gs_LOG = gs_LOG.fit(twenty_train['data'][:5000], twenty_train['target'][:5000])

In [ ]:
# print(gs_LOG.best_score_, gs_LOG.best_params_)
# LOGcv_predict = gs_LOG.predict(twenty_test['data'])
# print('LOG-CV accuracy: %.3f' %np.mean(LOGcv_predict, twenty_test['target']))
# print(metrics.classification_report(twenty_test['target'], SVMcv_predict))

# ANN

In [ ]:
from sklearn.neural_network import MLPClassifier
ANN_params = {'vect__ngram_range':[(1,1),(1,2)],
              'vect__analyzer':['word','char'],
              'tfidf__use_idf':('True'),
              'model__hidden_layer_sizes':[(3,), (3,3),(3,3,3)],
              'model__alpha':(1e-1, 1e-3)}

ANN_pipe = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('model',MLPClassifier(tol=1e-4, max_iter=200)),])

gs_ANN = GridSearchCV(ANN_pipe,
                      param_grid=ANN_params,
                      n_jobs=-1,
                      cv=4)

gs_ANN = gs_ANN.fit(twenty_train['data'][:5000], twenty_train['target'][:5000])

In [ ]:
print(gs_ANN.best_score_, gs_ANN.best_params_)
ANNcv_predict = gs_ANN.predict(twenty_test['data'])
print('ANN-CV accuracy: %.3f' %np.mean(ANNcv_predict, twenty_test['target']))
print(metrics.classification_report(twenty_test['target'], SVMcv_predict))

In [ ]:
# http://scikit-learn.org/stable/modules/model_evaluation.html#multilabel-ranking-metrics
# http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html#sklearn.metrics.classification_report

# Save Trained Models

In [ ]:
from sklearn.externals import joblib
model = r'trainedModels'
joblib.dump(gs_NB , model+'gs_NB_1.0.pkl') 
joblib.dump(gs_LOG, model+'gs_LOG_1.0.pkl') 
joblib.dump(gs_SVM, model+'gs_SVM_1.0.pkl') 
joblib.dump(gs_ANN, model+'gs_ANN_1.0.pkl') 

# trained-model = joblib.load('filename.pkl') 

# Results

In [ ]:
results = pd.DataFrame({'doc':X_test,
                        'target':y_test,
                        'NB':NBcv_predict,
                        'SVM':SVMcv_predict,
                        'LOG':LOGcv_predict,
                        'ANN':ANNcv_predict})

results = results[['doc','target', 'NB', 'SVM', 'LOG', 'ANN']].copy()
results.head()

# Score

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

In [ ]:
classes = list(y_test.unique())
classes

In [ ]:
Test = label_binarize(y_test, classes=classes)
Test.shape

In [ ]:
SVM = label_binarize(SVMcv_predict, classes=classes)
SVM.shape

In [ ]:
fig,axs = plt.subplots(1,1, figsize=(7,7))
fpr = dict()
tpr = dict()
roc_auc = dict()

for i,c in enumerate(classes):
    fpr[i], tpr[i], _ = roc_curve(Test[:, i], SVM[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    
    axs.plot(fpr[i], tpr[i], label='%s AUC:%.2f'%(c.upper(),roc_auc[i]))
    axs.legend()
    axs.set_xlim(0,1)
    axs.set_ylim(0,1)